# GMW Example

This notebook demonstrates how to register a groundwatermonitoringwell (GMW) in the BRO using bro-exchange.

In [10]:
import bro_exchange as brx
from getpass import getpass

Define the token used for authentification (Bronhouderportaal API) below. Note: a token generated in the demo environment should be used. Besides, the project id should be given when using api v2.

In [11]:
user = input('user:')
password = getpass('pass:')
project_id = input('project_id:')


## 1. Register a new GMW

### 1.1. Generate registration request

Get GMW data:

In [12]:
#%% Inputdata for monitoringtube elements

electrodes = [{'electrodeNumber':1,
               'electrodePackingMaterial':'zand',               
               'electrodeStatus':'gebruiksklaar',               
               'electrodePosition':-2.63,                              
               },
              {'electrodeNumber':2,
               'electrodePackingMaterial':'zand',               
               'electrodeStatus':'gebruiksklaar',               
               'electrodePosition':-3.98,                              
               }]


materialUsed_tube = {'tubePackingMaterial':'bentoniet',
                'tubeMaterial':'peHighDensity',
                'glue':'geen'} 

screen =  {'screenLength':1,
                'sockMaterial':'geen'} 

plainTubePart ={'plainTubePartLength':6.64}   

sedimentSump = {'sedimentSumpLength':0.5} 

geoOhmCables = [{'cableNumber':1,
               'electrodes':electrodes}]

monitoringTubes = [{'tubeNumber':1,
                    'tubeType':'standaardbuis',
                    'artesianWellCapPresent':'nee',
                    'sedimentSumpPresent':'ja',
                    'numberOfGeoOhmCables':1,
                    'tubeTopDiameter':32,
                    'variableDiameter':'nee',
                    'tubeStatus':'gebruiksklaar', 
                    'tubeTopPosition':3.42,
                    'tubeTopPositioningMethod':'RTKGPS10tot20cm',
                    'materialUsed':materialUsed_tube,
                    'screen':screen, 
                    'plainTubePart':plainTubePart},
                   {'tubeNumber':2,
                    'tubeType':'standaardbuis',
                    'artesianWellCapPresent':'nee',
                    'sedimentSumpPresent':'ja',
                    'numberOfGeoOhmCables':1,
                    'tubeTopDiameter':32,
                    'variableDiameter':'nee',
                    'tubeStatus':'gebruiksklaar', 
                    'tubeTopPosition':3.42,
                    'tubeTopPositioningMethod':'RTKGPS10tot20cm',
                    'materialUsed':materialUsed_tube,
                    'screen':screen, 
                    'plainTubePart':plainTubePart,
                    'sedimentSump':sedimentSump, 
                    'geoOhmCables':geoOhmCables}]

#%% Inputdata for GMW_Construction sourcedocument

deliveredLocation = {'X':134750.0,'Y':477800.0,'horizontalPositioningMethod':'DGPS50tot200cm'}

deliveredVerticalPosition = {'localVerticalReferencePoint':'NAP',
                                'offset':0,
                                'verticalDatum':'NAP',
                                'groundLevelPosition':3.18,
                                'groundLevelPositioningMethod':'RTKGPS10tot20cm'
                                }

srcdocdata =  {'objectIdAccountableParty':'B07F0076',
                'deliveryContext':'publiekeTaak',        
                'constructionStandard':'RWSgwmon',        
                'initialFunction':'stand',        
                'numberOfMonitoringTubes':2,        
                'groundLevelStable':'ja',        
                'wellStability':'instabiel',        
                'nitgCode':'B07F0076',        
                'owner':'27376655',        
                'maintenanceResponsibleParty':'27376655',        
                'wellHeadProtector':'geen',        
                'wellConstructionDate':'2015-02-08',        
                'deliveredLocation':deliveredLocation,        
                'deliveredVerticalPosition':deliveredVerticalPosition,             
                'monitoringTubes':monitoringTubes,              
                }    

Generate registration request (GMW_construction):

In [13]:
# Generate sourcedocument:
reg = brx.gmw_registration_request(srcdoc='GMW_Construction', requestReference = 'register - GMW_Construction - 018. IMBRO - groundwaterMonitoringWell.maintenanceResponsibleParty', deliveryAccountableParty = '52766179', qualityRegime = 'IMBRO', srcdocdata=srcdocdata)
reg.generate()

# Note: a file can be stored optionally, but it's not required
reg.write_request(filename = 'registration_request_gmw_construction_test.xml', output_dir = 'output\gmw')

### 1.2. Validate registration request

Validate the generated request. In this case, the demo environment of the bronhouderportaal is used for validation.

In [14]:
reg.validate(user=user, password = password, demo=True, api='v2', project_id = project_id)

Check if there are some errors:

In [15]:
print(reg.validation_status)
print(reg.validation_info)

NIET_VALIDE
{'status': 'NIET_VALIDE', 'errors': ['Grondwatermonitoringput.put stabiliteit (MonitoringWell.wellStability) mag niet aanwezig zijn:  Grondwatermonitoringput.maaiveld stabiel (MonitoringWell.groundLevelStable) = ja bepaalt dat het moet ontbreken.', 'Zandvang (SedimentSump) ontbreekt: het moet aanwezig zijn als Monitoringbuis. voorzien van zandvang (MonitoringTube.sedimentSumpPresent) = ja.', 'Geo-ohmkabel (GeoOhmCable) ontbreekt, {MonitoringWell.monitoringTube.numberOfGeoOhmCables} = 1 waarde bepaalt het aantal keer dat dit gegeven voorkomt.', 'Er zijn 1 of meer fouten geconstateerd in het brondocument.']}


The request isn't valid. Perform the required adjustments and validate the sourcedocument again:

In [17]:
monitoringTubes[0]['numberOfGeoOhmCables']=0
monitoringTubes[0]['sedimentSumpPresent']='nee'
srcdocdata['groundLevelStable']='nee'
srcdocdata['monitoringTubes']=monitoringTubes


reg = brx.gmw_registration_request(srcdoc='GMW_Construction', requestReference = 'register - GMW_Construction - 018. IMBRO - groundwaterMonitoringWell.maintenanceResponsibleParty', deliveryAccountableParty = '52766179', qualityRegime = 'IMBRO', srcdocdata=srcdocdata)
reg.generate()

reg.validate(user=user, password = password, demo=True, api='v2', project_id = project_id)
print(reg.validation_status)
print(reg.validation_info)

VALIDE
{'status': 'VALIDE'}


The xml document seems to be valid

### 1.3. Deliver registration request

Deliver the generared request. 

In [18]:
reg.deliver(user=user, password=password, api='v2', project_id = 1255, demo=True)
print(reg.delivery_info.json())
print(reg.delivery_id)

{'identifier': '0000182615', 'status': 'AANGELEVERD', 'lastChanged': '2022-11-15T23:24:54.318933', 'brondocuments': [{'id': 418603, 'status': 'GELDIG', 'lastChanged': '2022-11-15T23:24:54.347512', 'filename': '8bbcc2326a75.xml', 'requestReference': 'register - GMW_Construction - 018. IMBRO - groundwaterMonitoringWell.maintenanceResponsibleParty'}]}
0000182615


The request is delivered, and the id is stored. After delivering, the request can't be deliverd again:

In [19]:
reg.deliver(user=user, password=password, api='v2', project_id = 1255, demo=True)

Exception: Request has already been delivered

After the request has been deliverd, the delivery id can be stored to track the status.

In [21]:
delivery_id = reg.delivery_id

### 1.4. Track status of pending delivery

To track the status of a pending delivery, the 'check_status' function of bro-exchange can be used. It only requires the delivery id, api version, environment and the authentication data. If api v2 is used, the project id should also be supplied:

In [22]:
status = brx.check_delivery_status(delivery_id, user=user, password=password, api='v2', project_id = project_id, demo=True)
print(status.json())

{'identifier': '0000182615', 'status': 'DOORGELEVERD', 'lastChanged': '2022-11-15T23:24:58.310766', 'brondocuments': [{'id': 418603, 'status': 'OPGENOMEN_LVBRO', 'lastChanged': '2022-11-15T23:24:58.082592', 'broId': 'GMW000000042661', 'transactionId': 'GMW-000000046742', 'filename': '8bbcc2326a75.xml', 'requestReference': 'register - GMW_Construction - 018. IMBRO - groundwaterMonitoringWell.maintenanceResponsibleParty'}]}


Retrieve the broid:

In [23]:
broid_GMW = status.json()['brondocuments'][0]['broId']
print(broid_GMW)

GMW000000042661
